In [1]:
import glob, json

import sys
sys.path.append("/scr/BEHAVIOR-1K/asset_pipeline")

from b1k_pipeline.utils import get_targets, parse_name

In [ ]:
import pathlib

object_lists = {}
for target in get_targets("combined"):
    data = json.loads((pathlib.Path("/scr/BEHAVIOR-1K/asset_pipeline") / "cad" / target / "artifacts/object_list.json").read_text())
    object_lists[target] = data

In [3]:
from bddl.knowledge_base import *

objs_by_id = {obj.name.split("-")[-1]: obj for obj in Object.all_objects()}

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
import collections

male_attachments = collections.defaultdict(set)
female_attachments = collections.defaultdict(set)
for target, target_info in object_lists.items():
  for item_name, _, parent in target_info["max_tree"]:
    pn = parse_name(item_name)
    if pn is None:
      continue
    if pn.group("meta_type") != "attachment":
      continue
    attachment_type = pn.group("meta_id")
    # Correct known issues
    if attachment_type in attachment_group_fixes:
      attachment_type = attachment_group_fixes[attachment_type]

    # Validate and parse
    if len(attachment_type) == 0:
      print(f"Missing attachment type on object {parent} in target {target}")
      continue
    if attachment_type[-1] not in "MF":
      print(f"Invalid attachment type {attachment_type} on object {parent} in target {target}")
      continue
    is_attachment_female = attachment_type[-1] == "F"
    attachment_type = attachment_type[:-1]
    if len(attachment_type) == 0:
      print(f"Missing attachment type on object {parent} in target {target}")
      continue

    parent_id = parse_name(parent).group("model_id")
    obj = objs_by_id[parent_id]
    if is_attachment_female:
      female_attachments[attachment_type].add(obj)
    else:
      male_attachments[attachment_type].add(obj)

In [11]:
import collections, glob, pathlib, json
from tqdm.notebook import tqdm

male_attachments_out = collections.defaultdict(set)
female_attachments_out = collections.defaultdict(set)
all_metadata_files = list(glob.glob("/scr/BEHAVIOR-1K/asset_pipeline/tmp/export_objs2/objects/*/*/misc/metadata.json"))
for fn in tqdm(all_metadata_files):
  fn = pathlib.Path(fn)
  metadata = json.loads(fn.read_text())
  attachments = set()
  for link_mls in metadata["meta_links"].values():
    if "attachment" not in link_mls:
      continue
    attachments.update(k for k, v in link_mls["attachment"].items() if v)

  parent_id = fn.parts[-3]
  obj = objs_by_id[parent_id]

  for attachment_type in attachments:
    # Validate and parse
    if len(attachment_type) == 0:
      print(f"Missing attachment type on object {parent_id}")
      continue
    if attachment_type[-1] not in "MF":
      print(f"Invalid attachment type {attachment_type} on object {parent_id}")
      continue
    is_attachment_female = attachment_type[-1] == "F"
    attachment_type = attachment_type[:-1]
    if len(attachment_type) == 0:
      print(f"Missing attachment type on object {parent_id}")
      continue

    if is_attachment_female:
      female_attachments_out[attachment_type].add(obj)
    else:
      male_attachments_out[attachment_type].add(obj)

  0%|          | 0/9219 [00:00<?, ?it/s]

In [13]:
# Check that every attachment has a male and a female end
print("Missing female end:", set(male_attachments_out) - set(female_attachments_out))
print("Missing male end:", set(female_attachments_out) - set(male_attachments_out))

Missing female end: {'duweraparent'}
Missing male end: set()


In [20]:
# Combinatorially generate every pair of objects that can be attached to each other
attachable_objects = set()
for attachment_type in set(male_attachments_out) | set(female_attachments_out):
  for male_candidate in male_attachments_out[attachment_type]:
    for female_candidate in female_attachments_out[attachment_type]:
      attachable_objects.add((male_candidate.name, female_candidate.name))

attachable_objects = sorted(attachable_objects)

with open("/scr/BEHAVIOR-1K/asset_pipeline/metadata/attachment_combinations.json", "w") as f:
  json.dump(attachable_objects, f)

attachable_objects

[('address-pbfbxj', 'recreational_vehicle-doiucd'),
 ('address-pbfbxj', 'wall_nail-wlnail'),
 ('antlers-nlsanr', 'recreational_vehicle-doiucd'),
 ('antlers-nlsanr', 'wall_nail-wlnail'),
 ('bicycle-xjumcf', 'bicycle_rack-mltrbs'),
 ('bicycle_rack-mltrbs', 'recreational_vehicle-doiucd'),
 ('bicycle_rack-mltrbs', 'wall_nail-wlnail'),
 ('blackberry-tvpfvb', 'bush-waafqw'),
 ('bow-fhchql', 'recreational_vehicle-doiucd'),
 ('bow-fhchql', 'wall_nail-wlnail'),
 ('broken_light_bulb-cugtye', 'table_lamp-ehjsdz'),
 ('cabinet_door-ertkre', 'cabinet_base-dcmhhh'),
 ('cap-actmgl', 'jar_of_strawberry_jam-busiti'),
 ('cap-amryfj', 'bottle_of_water-qaceen'),
 ('cap-arskxc', 'bottle_of_whiskey-lzjuvy'),
 ('cap-ceizxn', 'bottle_of_strawberry_juice-mlnuza'),
 ('cap-ciopwh', 'jar_of_kidney_beans-kfzxah'),
 ('cap-cjtyvz', 'blender-xnjqix'),
 ('cap-clhquh', 'jar_of_pepper_seasoning-iuydyz'),
 ('cap-dduopd', 'jar-pjaljg'),
 ('cap-dmzavi', 'pill_bottle-wsasmm'),
 ('cap-dwpcld', 'jar_of_clove-dlvall'),
 ('cap-e